# Задание 1

### Импортируем модули для разработки автокодировщиков

In [ ]:
# Подключим Numpy
import numpy as np

# Подключим библиотеку отображения графиков
import matplotlib.pyplot as plt

# Подключим датасет рукописных цифр MNIST
from tensorflow.keras.datasets import mnist

# Импортируем Keras
from tensorflow import keras

# Подключим все необходимые слои Keras
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Lambda, LeakyReLU
from tensorflow.keras.layers import BatchNormalization, Dropout, concatenate, Conv2D, Conv2DTranspose

# Подключим модуль вычислений на Keras
import keras.backend as K

# Задание 2

Создадим вариационный автокодировщик для генерации рукописных цифр. В качестве обучающего датасета возьмем набор цифр MNIST. Размер изображнения 28х28, градации серого. Допустим, что изображения уже подготовлены, а именно выполнен решейп изображений, значения пикселей приведены к диапазону [0...1]. В работе пользуйтесь техникой функционального программирования. Размерность скрытого пространства =3

## 2.1 Создайте энкодер

Напишем входной слой энкодера. Назовем вход энкодера - "input_img". Размерность входного тензора (28, 28, 1)

In [ ]:
input_img = Input(shape=(28, 28, 1)) 

Добавим слой Flatten

In [ ]:
x = Flatten()(input_img)  

Добавим Dense слой с параметрами: 256 нейронов, функция активации "relu"

In [ ]:
x = Dense(256,activation='relu')(x)

Добавим Dense слой с параметрами: 16 нейронов, функция активации "relu"

In [ ]:
x = Dense(16,activation='relu')(x)

Добавим выходной Dense слой с параметрами число нейронов равно размерности скрытого постранства, функция активации "linear". Назовем выход энкодера - h

In [ ]:
h = Dense(3,activation='linear')(x)

Подключим параллельно ко выходу энкодера 2 dense слоя для оценки мат.ожидания и логарифма дисперсии. Выходы слоев назовем z_mean и z_log_var.

In [ ]:
z_mean = Dense(3)(h)       
z_log_var = Dense(3)(h)     

Создадим функцию - генератор тензоров из нормальных случайных чисел с заданным мат.ожидаением и дисперсией. Функцию принимает на вход список параметров args. 
Функция назовем noiser(). Функция содержит 4 строки. Далее построчно:

1. Разделим список args на отдельные параметры z_mean и z_log_var
2. Заполним случайный тензор размерностью [K.shape(z_mean)[0], hidden dim] = [32,3] нормальным случайными числами с нулевым мат.ожидание и единичной дисперсией
3. Изменим дисперсию чисел в тензоре домножением на K.exp(z_log_var / 2), добавим мат.ожидание. Здесь необходимо пользоваться математикой от keras- backend.
4. Вернем полученный тензор

In [ ]:
def noiser(args):
  z_mean, z_log_var = args
  N = K.random_normal(shape=(K.shape(z_mean)[0], 3), mean=0., stddev=1.0)
  N = K.exp(z_log_var / 2) * N + z_mean 
  return N

Напишем lambda слой - генератор случайных тензоров. На вход принимает список [z_mean, z_log_var], исполняемая функия noiser. Выход функции назовем h

In [ ]:
h = Lambda(noiser, output_shape=(3,))([z_mean, z_log_var])

Соберем модель энкодера методом keras.Model(). Вход - input_img. Выходы - [z_mean, z_log_var, h]

In [ ]:
encoder = keras.Model(input_img, [z_mean, z_log_var, h]) 

## 2.2 Создайте декодер

Напишем входной слой Input декодера. Назовем вход - "input_enc". Размерность входного тензора равна размерность скрытого пространства - (3,)

In [ ]:
input_enc = Input(shape=(3,))  

Добавим Dense слой с параметрами: 16 нейронов, функция активации "relu"

In [ ]:
x = Dense(16,activation='relu')(input_enc)

Добавим Dense слой с параметрами: 784 нейронов, функция активации "relu"

In [ ]:
x = Dense(784,activation='relu')(x)

Выполним изменение размерности тензора до исходной - (28, 28, 1). Выход слоя назовите "decoded"

In [ ]:
decoded = Reshape((28, 28, 1))(x)

Соберем модель декодера keras.Model(). Вход "input_enc', выход "decoded".

In [ ]:
decoder = keras.Model(input_enc, decoded)  

Соберем модель AE целиком, из готового энкодера и декодера. Не забудем, что на вход декодера подается только один, а именно последный выход энкодера. Для сборки используем метод keras.Model()

In [ ]:
autoencoder = keras.Model(input_img, decoder(encoder(input_img)[2]))

С помощью метода .summary() выведем на экран структуру энкодера.

In [ ]:
encoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 784)          0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          200960      flatten[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           4112        dense[0][0]                      
______________________________________________________________________________________________

С помощью метода .summary() выведем на экран структуру декодера.

In [ ]:
decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                64        
_________________________________________________________________
dense_6 (Dense)              (None, 784)               13328     
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
Total params: 13,392
Trainable params: 13,392
Non-trainable params: 0
_________________________________________________________________


# Задание 3

Создадим сверточный автокодирощик с условием для генерации цветных изображений. В качестве обучающего датасета возьмем CIFAR-10. Размер изображнения 32х32, цвет, 10 классов. Допустим, что изображения уже подготовлены,  значения пикселей приведены к диапазону [0...1]. Метки классов преобразованы в one-hot encoding. В работе пользуйемся техникой функционального программирования. Размерность скрытого пространства =8

## 3.1 Создадим энкодер

Напишем входной слой энкодера дя изображения. Назовем вход энкодера - input_img. Размерность входного тензора (32, 32,  3)

In [ ]:
input_img = Input(shape=(32, 32, 3))

Добавим сверточный слой. Число нейронов - 128, ядро - 3, сохраняем размер слоя

In [ ]:
x = Conv2D(128, kernel_size=3, padding='same')(input_img)

Добавим сверточный слой. Число нейронов - 64, ядро - 3, сохраняем размер слоя

In [ ]:
x = Conv2D(64, kernel_size=3, padding='same')(x)

Добавим 1 слой Flatten. Выход слоя назовем x

In [ ]:
x = Flatten()(x)  

Напишем входной слой энкодера для метки класса. Назовем вход энкодера - lb. Размерность входного тензора (10,)

In [ ]:
lb = Input(shape=(10,)) 

Добавим слой конкатенации x и lb

In [ ]:
x = concatenate([x, lb])

Добавим Dense слой с параметрами: 125 нейронов, функция активации "relu"

In [ ]:
x = Dense(125,activation='relu')(x)

Добавим Dense слой с параметрами: 8 нейронов, функция активации "relu"

In [ ]:
x = Dense(8, activation='relu')(x)

Подключим к выходу энкодера 2 dense слоя для оценки мат.ожидания и логарифма дисперсии. Выходы слоев назовем z_mean и z_log_var.

In [ ]:
z_mean = Dense(8)(x)       
z_log_var = Dense(8)(x)  

Создадим функцию - генератор тензоров из нормальных случайных чисел с заданным мат.ожидаением и дисперсией. Функцию принимает на вход список параметров args. 
Функция назовем noiser(). Функция содержит 4 строки. Далее построчно:

1. Разделим список args на отдельные параметры z_mean и z_log_var
2. Заполним случайный тензор размерностью [K.shape(z_mean)[0], hidden dim] = [32,3] нормальным случайными числами с нулевым мат.ожидание и единичной дисперсией
3. Изменим дисперсию чисел в тензоре домножением на K.exp(z_log_var / 2), добавим мат.ожидание. Здесь необходимо пользоваться математикой от keras- backend.
4. Вернем полученный тензор

In [ ]:
def noiser(args):
  z_mean, z_log_var = args
  N = K.random_normal(shape=(K.shape(z_mean)[0], 8), mean=0., stddev=1.0)
  N = K.exp(z_log_var / 2) * N + z_mean 
  return N

Напишем lambda слой - генератор случайных тензоров. На ход принимает список [z_mean, z_log_var], исполняемая функия noiser. Выход слоя назовем h

In [ ]:
h = Lambda(noiser, output_shape=(8,))([z_mean, z_log_var])

Соберем модель энкодера методом keras.Model(). Вход - input_img. Выходы - [z_mean, z_log_var, h]

In [ ]:
encoder = keras.Model([input_img, lb ], [z_mean, z_log_var, h]) 

## 3.2 Создадим декодер

Напишем входной слой Input декодера для точек скрытого пространства. Назовем вход - "input_enc". Размерность входного тензора равна размерность скрытого пространства - (8,)

In [ ]:
input_enc = Input(shape=(8,))  

Напишем входной слой Input декодера для меток класса. Назовем вход - "lb_dec ". Размерность входного тензора равна числу классов - (10,)

In [ ]:
lb_dec = Input(shape=(10,)) 

Добавим слой конкатенации input_enc и lb_dec



In [ ]:
x = concatenate([input_enc, lb_dec])

Добавим Dense слой с параметрами: 2048 нейронов, функция активации "relu"

In [ ]:
x = Dense(2048 ,activation='relu')(x)

Выполним решейп до размерности [8,8,32]

In [ ]:
x = Reshape((8, 8, 32))(x) 

Добавим обратную свертку с параметрами: число нейронов 64, ядро 3, шаг = 2, padding='same', функция активации 'relu'

In [ ]:
x = Conv2DTranspose(64, kernel_size=3, strides= 2, padding='same',  activation='relu')(x) 

Добавим обратную свертку с параметрами: число нейронов 64, ядро 3, шаг = 2, padding='same', функция активации 'relu'

In [ ]:
decoded = Conv2DTranspose(3, kernel_size=3, strides= 2, padding='same',  activation='relu')(x)

Соберем декодер. Входы input_enc и lb_dec. Выход decoded.




In [ ]:
decoder = keras.Model([input_enc , lb_dec], decoded)  

Соберем CVAE. Входы input_img, lb, lb_dec. Выход decoded . У энкодера необходимо взять 2й выход и подать на декодер


In [ ]:
cvae = keras.Model([input_img, lb, lb_dec], decoder([encoder([input_img, lb])[2], lb_dec])) 

С помощью метода .summary() выведем на экран структуру энкодера.

In [ ]:
encoder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 128)  3584        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   73792       conv2d[0][0]                     
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 65536)        0           conv2d_1[0][0]                   
____________________________________________________________________________________________

С помощью метода .summary() выведем на экран структуру декодера.

In [ ]:
decoder.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 18)           0           input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2048)         38912       concatenate_1[0][0]        